### Funcionou (5% da anterior, bonificações sobre movimentos corretos e modo roleta)

In [1]:
import random
import numpy as np
import pandas as pd

# Constantes de Estabilidade
KM = 12  # Altura do metacentro acima da quilha em metros
KG = 8   # Altura inicial do centro de gravidade em metros
MIN_GM = 0.5  # Mínimo valor permitido para GM em metros
MAX_HEEL = 5  # Máximo valor permitido para inclinação lateral em graus
MAX_TRIM = 2  # Máximo valor permitido para inclinação longitudinal em graus
CAPACIDADE_MAXIMA = 550  # Capacidade máxima da balsa em toneladas

# Função para calcular a estabilidade
def calcular_estabilidade(containers):
    peso_total = sum(peso for peso, _, _, _ in containers)
    centro_gravidade_z = sum(peso * z for peso, _, _, z in containers) / peso_total
    GM = KM - centro_gravidade_z
    heel = sum(peso * x for peso, x, _, _ in containers) / (peso_total * 10)
    trim = sum(peso * y for peso, _, y, _ in containers) / (peso_total * 10)
    
    if GM < MIN_GM or abs(heel) > MAX_HEEL or abs(trim) > MAX_TRIM:
        return False, GM, heel, trim
    return True, GM, heel, trim

# Função modificada para incluir cálculo de estabilidade
def preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    cal_movimentos = 0
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    containers = []
    
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final'] 
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        errou = 0
        pode_mover = True

        if (errou==0):
            
            # Verificar se podemos remover do porto preenchido
            if andar_ini < 1:
                if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                    errou =1
                    pode_mover = False
            
            # Verificar se podemos adicionar à balsa
            if andar_fin > 0:
                if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                    errou =1
                    pode_mover = False

            # Verifica se o lugar está ocupado
            if balsa[andar_fin , x_fin, y_fin] == 1:
                errou =1
                pode_mover = False

                    
            if pode_mover:
                # Atualizar porto_preenchido e balsa
                porto_preenchido[andar_ini, x_ini, y_ini] = 0
                balsa[andar_fin, x_fin, y_fin] = 1
                
                cal_movimentos += 2 #alturas subir e descer
                cal_movimentos += 0 if x_ini == x_fin else 1  # verificação x
                cal_movimentos += 0 if y_ini == y_fin else 1  # verificação y
                
                # Adicionar contêiner à lista
                peso_container = 10  # Supondo peso fixo de 10 toneladas por contêiner
                containers.append((peso_container, x_fin, y_fin, 1))  # Altura simplificada

                estavel, GM, heel, trim = calcular_estabilidade(containers)
                if not estavel:
                    errou =1
                    pode_mover = False # Penalidade para cromossomos instáveis
            else:
                cal_movimentos += 1000
        else:
            cal_movimentos += 1000

    
    return cal_movimentos

# Funções do algoritmo genético (permanecem iguais, apenas chamam a função modificada)
def gerar_cromossomo():
    cromossomo = []
    
    # Gera posições iniciais (0-29)
    posicoes_iniciais = list(range(30))
    random.shuffle(posicoes_iniciais)
    cromossomo.extend(posicoes_iniciais)
    
    # Gera posições finais (0-29)
    posicoes_finais = list(range(30))
    random.shuffle(posicoes_finais)
    cromossomo.extend(posicoes_finais)
    
    # Gera ordem de movimentação (0-29)
    ordem_movimentacao = list(range(30))
    random.shuffle(ordem_movimentacao)
    cromossomo.extend(ordem_movimentacao)
    
    return cromossomo

def cromossomo_para_dataframe(cromossomo):
    container = list(range(30))
    posicao_inicial = cromossomo[:30]
    posicao_final = cromossomo[30:60]
    ordem_movimentacao = cromossomo[60:90]
    
    df = pd.DataFrame({
        'Container': container,
        'Posição Inicial': posicao_inicial,
        'Posição Final': posicao_final,
        'Ordem de Movimentação': ordem_movimentacao
    })
        
    df = df.sort_values(by='Ordem de Movimentação')
    
    return df

def inicializar_populacao(tamanho_populacao):
    populacao = []
    for _ in range(tamanho_populacao):
        cromossomo = gerar_cromossomo()
        populacao.append(cromossomo)
    return populacao

def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

def selecao_roleta(populacao, fitness):
    soma_fitness = sum(fitness)
    pick = random.uniform(0, soma_fitness)
    atual = 0
    for cromossomo, fit in zip(populacao, fitness):
        atual += fit
        if atual >= pick:
            return cromossomo

def cruzamento_pmx(pai1, pai2, inicio, fim):
    tamanho = fim - inicio + 1
    filho1 = pai1[:inicio] + [-1] * tamanho + pai1[fim+1:]
    filho2 = pai2[:inicio] + [-1] * tamanho + pai2[fim+1:]
    
    mapeamento1 = pai1[inicio:fim+1]
    mapeamento2 = pai2[inicio:fim+1]
    
    # Copiar o segmento
    filho1[inicio:fim+1] = mapeamento2
    filho2[inicio:fim+1] = mapeamento1
    
    # Resolver conflitos para filho1
    for i in range(inicio):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    for i in range(fim+1, len(pai1)):
        if filho1[i] in mapeamento2:
            idx = mapeamento2.index(filho1[i])
            while mapeamento1[idx] in mapeamento2:
                idx = mapeamento2.index(mapeamento1[idx])
            filho1[i] = mapeamento1[idx]
    
    # Resolver conflitos para filho2
    for i in range(inicio):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    for i in range(fim+1, len(pai2)):
        if filho2[i] in mapeamento1:
            idx = mapeamento1.index(filho2[i])
            while mapeamento2[idx] in mapeamento1:
                idx = mapeamento1.index(mapeamento2[idx])
            filho2[i] = mapeamento2[idx]
    
    return filho1, filho2

def cruzamento(pai, mae):
    filho1, filho2 = pai[:], mae[:]
    
    ponto_corte1 = random.randint(0, 29)
    ponto_corte2 = random.randint(0, 29)
    inicio, fim = min(ponto_corte1, ponto_corte2), max(ponto_corte1, ponto_corte2)
    filho1[:30], filho2[:30] = cruzamento_pmx(pai[:30], mae[:30], inicio, fim)
    
    ponto_corte1 = random.randint(30, 59)
    ponto_corte2 = random.randint(30, 59)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 30, max(ponto_corte1, ponto_corte2) - 30
    filho1[30:60], filho2[30:60] = cruzamento_pmx(pai[30:60], mae[30:60], inicio, fim)
    
    ponto_corte1 = random.randint(60, 89)
    ponto_corte2 = random.randint(60, 89)
    inicio, fim = min(ponto_corte1, ponto_corte2) - 60, max(ponto_corte1, ponto_corte2) - 60
    filho1[60:90], filho2[60:90] = cruzamento_pmx(pai[60:90], mae[60:90], inicio, fim)
    
    return filho1, filho2

def mutacao(cromossomo, taxa_mutacao=0.01):
    if random.random() < taxa_mutacao:
        part = random.randint(0, 2)
        idx1, idx2 = random.sample(range(30), 2)
        if part == 0:
            cromossomo[idx1], cromossomo[idx2] = cromossomo[idx2], cromossomo[idx1]
        elif part == 1:
            cromossomo[30+idx1], cromossomo[30+idx2] = cromossomo[30+idx2], cromossomo[30+idx1]
        else:
            cromossomo[60+idx1], cromossomo[60+idx2] = cromossomo[60+idx2], cromossomo[60+idx1]
    return cromossomo

def algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao):
    populacao = inicializar_populacao(tamanho_populacao)
    fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
    
    for geracao in range(geracoes):
        nova_populacao = []
        while len(nova_populacao) < tamanho_populacao:
            pai = selecao_roleta(populacao, fitness)
            mae = selecao_roleta(populacao, fitness)
            
            filho1, filho2 = cruzamento(pai, mae)
            filho1 = mutacao(filho1, taxa_mutacao)
            filho2 = mutacao(filho2, taxa_mutacao)
            
            nova_populacao.append(filho1)
            nova_populacao.append(filho2)
        
        # Combinar 5% melhor da geração antiga com a melhor nova geração
        populacao.extend(nova_populacao)
        fitness = [preencher_porto(cromossomo_para_dataframe(cromossomo)) for cromossomo in populacao]
        
        # Ordenar pela fitness e selecionar os melhores
        populacao_ordenada = sorted(zip(populacao, fitness), key=lambda x: x[1])
        
        melhor_5_porcento = int(tamanho_populacao * 0.05)
        nova_metade = tamanho_populacao - melhor_5_porcento
        
        populacao = [cromossomo for cromossomo, _ in populacao_ordenada[:melhor_5_porcento]] + [cromossomo for cromossomo, _ in populacao_ordenada[melhor_5_porcento:melhor_5_porcento+nova_metade]]
        fitness = [fit for _, fit in populacao_ordenada[:melhor_5_porcento]] + [fit for _, fit in populacao_ordenada[melhor_5_porcento:melhor_5_porcento+nova_metade]]
        
        melhor_fit = min(fitness)
        print(f"Geração {geracao+1}: Melhor fitness = {melhor_fit}")
    
    melhor_cromossomo = populacao[fitness.index(min(fitness))]
    return melhor_cromossomo, min(fitness)

# Configurações
tamanho_populacao = 100
geracoes = 200
taxa_mutacao = 0.1

melhor_cromossomo, melhor_fit = algoritmo_genetico(tamanho_populacao, geracoes, taxa_mutacao)
print(f"Melhor cromossomo encontrado: {melhor_cromossomo}")
print(f"Melhor fitness: {melhor_fit}")


Geração 1: Melhor fitness = 11070
Geração 2: Melhor fitness = 11070
Geração 3: Melhor fitness = 11070
Geração 4: Melhor fitness = 11063
Geração 5: Melhor fitness = 11063
Geração 6: Melhor fitness = 11063
Geração 7: Melhor fitness = 11063
Geração 8: Melhor fitness = 9073
Geração 9: Melhor fitness = 9073
Geração 10: Melhor fitness = 9073
Geração 11: Melhor fitness = 9073
Geração 12: Melhor fitness = 9073
Geração 13: Melhor fitness = 9073
Geração 14: Melhor fitness = 9073
Geração 15: Melhor fitness = 7081
Geração 16: Melhor fitness = 7081
Geração 17: Melhor fitness = 7081
Geração 18: Melhor fitness = 7081
Geração 19: Melhor fitness = 7081
Geração 20: Melhor fitness = 7081
Geração 21: Melhor fitness = 7081
Geração 22: Melhor fitness = 7081
Geração 23: Melhor fitness = 7081
Geração 24: Melhor fitness = 7081
Geração 25: Melhor fitness = 7081
Geração 26: Melhor fitness = 5089
Geração 27: Melhor fitness = 5089
Geração 28: Melhor fitness = 5089
Geração 29: Melhor fitness = 5089
Geração 30: Melh

#### Verifica se ha numeros repetidos


In [2]:
def verifica_repetidos(cromossomo):
    intervalos = [
        (0, 29),
        (30, 59),
        (60, 89)
    ]
    
    for intervalo in intervalos:
        valores = cromossomo[intervalo[0]:intervalo[1]+1]
        if len(valores) != len(set(valores)):
            return True
    
    return False

if verifica_repetidos(melhor_cromossomo):
    print("Valores repetidos encontrados!")
else:
    print("Não há valores repetidos.")

df_cromossomo = cromossomo_para_dataframe(melhor_cromossomo)
print("DataFrame do Cromossomo:")
print(df_cromossomo)


Não há valores repetidos.
DataFrame do Cromossomo:
    Container  Posição Inicial  Posição Final  Ordem de Movimentação
12         12               17              0                      0
3           3               27              7                      1
1           1               23              8                      2
14         14               28             10                      3
8           8               13             22                      4
21         21               19              5                      5
10         10               16              1                      6
29         29                1             13                      7
24         24               26             17                      8
11         11                2              2                      9
26         26               21              6                     10
0           0               12              4                     11
5           5                8             11       

### Simular Melhor cromomossomo

In [3]:
def Simular_preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    cal_movimentos = 0
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1
    
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')
    containers = []
    
    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final'] 
        
        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3
        
        errou = 0
        pode_mover = True

        if (errou==0):
            
            # Verificar se podemos remover do porto preenchido
            if andar_ini < 1:
                if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                    errou =1
                    pode_mover = False
            
            # Verificar se podemos adicionar à balsa
            if andar_fin > 0:
                if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                    errou =1
                    pode_mover = False

            # Verifica se o lugar está ocupado
            if balsa[andar_fin , x_fin, y_fin] == 1:
                errou =1
                pode_mover = False

                    
            if pode_mover:
                # Atualizar porto_preenchido e balsa
                porto_preenchido[andar_ini, x_ini, y_ini] = 0
                balsa[andar_fin, x_fin, y_fin] = 1
                
                cal_movimentos += 2 #alturas subir e descer
                cal_movimentos += 0 if x_ini == x_fin else 1  # verificação x
                cal_movimentos += 0 if y_ini == y_fin else 1  # verificação y
                
                # Adicionar contêiner à lista
                peso_container = 10  # Supondo peso fixo de 10 toneladas por contêiner
                containers.append((peso_container, x_fin, y_fin, 1))  # Altura simplificada

                estavel, GM, heel, trim = calcular_estabilidade(containers)

                print(f"Movendo container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")
                print(f"Estabilidade {estavel}: GM->({GM}) heel->({heel}) trim->({trim})")
                print("Estado atual do Porto Preenchido:")
                print(porto_preenchido)
                print("Estado atual da Balsa:")
                print(balsa)
                print("\n")

                if not estavel:
                    errou =1
                    pode_mover = False # Penalidade para cromossomos instáveis
            else:
                print(f"não moveu container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")
                cal_movimentos += 1000
        else:
            cal_movimentos += 1000


Simular_preencher_porto(cromossomo_para_dataframe(melhor_cromossomo))

Movendo container 12 de (17)(17)(1, 0, 2) para (0)(0)(0, 0, 0)
Estabilidade True: GM->(11.0) heel->(0.0) trim->(0.0)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 0]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]]
Estado atual da Balsa:
[[[1 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 3 de (27)(27)(1, 4, 0) para (7)(7)(0, 2, 1)
Estabilidade True: GM->(11.0) heel->(0.1) trim->(0.05)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [1 1 1]]

 [[1 1 0]
  [1 1 1]
  [1 1 1]
  [1 1 1]
  [0 1 1]]]
Estado atual da Balsa:
[[[1 0 0]
  [0 0 0]
  [0 1 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  [0 0 0]]]


Movendo container 1 de (23)(23)(1, 2, 2) para (8)(8)(0, 2, 2)
Estabilidade True: GM->(11.0) heel->(0.13333333333333333) trim->(0.1)
Estado atual do Porto Preenchido:
[[[1 1 1]
  [1 1 1]
  

#### Estabilidade

In [1]:
# Constantes
KM = 12  # Altura do metacentro acima da quilha em metros
KG = 8   # Altura inicial do centro de gravidade em metros
MIN_GM = 0.5  # Mínimo valor permitido para GM em metros
MAX_HEEL = 5  # Máximo valor permitido para inclinação lateral em graus
MAX_TRIM = 2  # Máximo valor permitido para inclinação longitudinal em graus
CAPACIDADE_MAXIMA = 550  # Capacidade máxima da balsa em toneladas

# Variáveis iniciais
peso_total = 0  # Peso total inicial da balsa em toneladas
centro_gravidade_z = KG  # Centro de gravidade inicial

# Lista para armazenar os contêineres (cada contêiner é uma tupla (peso, x, y, z))
containers = []

def adicionar_container(peso, x, y, z):
    global peso_total, centro_gravidade_z, KM
    if peso_total + peso > CAPACIDADE_MAXIMA:
        print("Erro: Capacidade máxima da balsa excedida.")
        return

    containers.append((peso, x, y, z))
    
    # Atualize o peso total
    peso_total_novo = peso_total + peso
    
    # Calcule o novo centro de gravidade (G_new)
    centro_gravidade_z_novo = (centro_gravidade_z * peso_total + z * peso) / peso_total_novo
    
    # Atualize o peso total
    peso_total = peso_total_novo
    centro_gravidade_z = centro_gravidade_z_novo
    
    # Calcule GM
    GM = KM - centro_gravidade_z
    
    # Verifique a inclinação lateral (heel) e a inclinação longitudinal (trim)
    heel = calcular_heel(containers)
    trim = calcular_trim(containers)
    
    # Verifique a estabilidade
    if not esta_estavel(GM, heel, trim):
        print("Ajuste necessário para manter a estabilidade.")
        ajustar_containers()
        # Recalcular os valores após o ajuste
        GM = KM - centro_gravidade_z
        heel = calcular_heel(containers)
        trim = calcular_trim(containers)

    print(f"Contêiner adicionado. GM: {GM:.2f} metros, Heel: {heel:.2f} graus, Trim: {trim:.2f} graus")

def calcular_heel(containers):
    # Calcular o momento lateral (momento em torno do eixo longitudinal da balsa)
    momento_lateral = sum(peso * x for peso, x, y, z in containers)
    # Simplificação para cálculo da inclinação lateral em graus
    heel = momento_lateral / (peso_total * 10)  # Dividido por um fator arbitrário para conversão
    return heel

def calcular_trim(containers):
    # Calcular o momento longitudinal (momento em torno do eixo lateral da balsa)
    momento_longitudinal = sum(peso * y for peso, x, y, z in containers)
    # Simplificação para cálculo da inclinação longitudinal em graus
    trim = momento_longitudinal / (peso_total * 10)  # Dividido por um fator arbitrário para conversão
    return trim

def esta_estavel(GM, heel, trim):
    # Verifique se GM, heel, e trim estão dentro dos limites permitidos
    if GM < MIN_GM or abs(heel) > MAX_HEEL or abs(trim) > MAX_TRIM:
        return False
    return True

def ajustar_containers():
    # Lógica simplificada para ajustar os contêineres: realocar para balancear melhor
    global peso_total, centro_gravidade_z
    print("Rearranjando contêineres para manter a estabilidade...")
    peso_total = 0
    momento_lateral = 0
    momento_longitudinal = 0
    centro_gravidade_z = 0
    for i, (peso, x, y, z) in enumerate(containers):
        # Realoca contêineres centralmente para balanceamento (ajuste simples)
        x = 0
        y = 0
        containers[i] = (peso, x, y, z)
        peso_total += peso
        momento_lateral += peso * x
        momento_longitudinal += peso * y
        centro_gravidade_z += z * peso
    
    centro_gravidade_z /= peso_total

# Exemplo de uso
adicionar_container(10, 5, 10, 15)  # Adiciona um contêiner de peso 10 toneladas na posição (5, 10, 15)
adicionar_container(8, -3, 8, 10)   # Adiciona um contêiner de peso 8 toneladas na posição (-3, 8, 10)
adicionar_container(15, 2, -5, 20)  # Adiciona um contêiner de peso 15 toneladas na posição (2, -5, 20)


Ajuste necessário para manter a estabilidade.
Rearranjando contêineres para manter a estabilidade...
Contêiner adicionado. GM: -3.00 metros, Heel: 0.00 graus, Trim: 0.00 graus
Ajuste necessário para manter a estabilidade.
Rearranjando contêineres para manter a estabilidade...
Contêiner adicionado. GM: -0.78 metros, Heel: 0.00 graus, Trim: 0.00 graus
Ajuste necessário para manter a estabilidade.
Rearranjando contêineres para manter a estabilidade...
Contêiner adicionado. GM: -4.06 metros, Heel: 0.00 graus, Trim: 0.00 graus


#### Animação

In [5]:
import pygame
import numpy as np
import pandas as pd

# Configurações iniciais
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
CELL_SIZE = 40
PORTO_POS = (50, 50)
BALSA_POS = (350, 50)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
FONT_SIZE = 24
BUTTON_COLOR = (0, 255, 0)
BUTTON_HOVER_COLOR = (0, 200, 0)
BUTTON_POSITION = (350, 500)
BUTTON_SIZE = (100, 50)

# Inicializar Pygame
pygame.init()
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption('Simulação de Preenchimento de Porto')

# Função para desenhar uma célula
def draw_cell(screen, color, pos, text):
    rect = pygame.Rect(pos[0], pos[1], CELL_SIZE, CELL_SIZE)
    pygame.draw.rect(screen, color, rect)
    font = pygame.font.SysFont(None, FONT_SIZE)
    img = font.render(text, True, BLACK)
    screen.blit(img, (pos[0] + 10, pos[1] + 10))

# Função para desenhar o porto e a balsa
def draw_porto_balsa(porto, balsa):
    screen.fill(WHITE)
    for andar in range(porto.shape[0]):
        for x in range(porto.shape[1]):
            for y in range(porto.shape[2]):
                pos = (PORTO_POS[0] + y * CELL_SIZE, PORTO_POS[1] + x * CELL_SIZE + andar * 150)
                draw_cell(screen, BLUE if porto[andar, x, y] > 0 else WHITE, pos, str(porto[andar, x, y]))

    for andar in range(balsa.shape[0]):
        for x in range(balsa.shape[1]):
            for y in range(balsa.shape[2]):
                pos = (BALSA_POS[0] + y * CELL_SIZE, BALSA_POS[1] + x * CELL_SIZE + andar * 150)
                draw_cell(screen, BLUE if balsa[andar, x, y] > 0 else WHITE, pos, str(balsa[andar, x, y]))

# Função para exibir texto na tela
def display_text(screen, text, position):
    font = pygame.font.SysFont(None, FONT_SIZE)
    img = font.render(text, True, BLACK)
    screen.blit(img, position)

# Função para desenhar um botão
def draw_button(screen, position, size, text, hover=False):
    color = BUTTON_HOVER_COLOR if hover else BUTTON_COLOR
    rect = pygame.Rect(position[0], position[1], size[0], size[1])
    pygame.draw.rect(screen, color, rect)
    font = pygame.font.SysFont(None, FONT_SIZE)
    img = font.render(text, True, BLACK)
    screen.blit(img, (position[0] + 10, position[1] + 10))

# Função de simulação
def Simular_preencher_porto(df_cromossomo, porto, balsa, move_next, move_index):
    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')

    if move_next and move_index < len(df_cromossomo):
        linha = df_cromossomo.iloc[move_index]
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial']
        pos_final = linha['Posição Final']

        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3

        # Atualizar o estado do porto e da balsa
        porto[andar_ini, x_ini, y_ini] = 0
        balsa[andar_fin, x_fin, y_fin] = container

        move_index += 1

    # Desenhar o estado atual do porto e da balsa
    draw_porto_balsa(porto, balsa)
    if move_index > 0:
        linha = df_cromossomo.iloc[move_index - 1]
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        display_text(screen, f'Movendo container {container} de {a_pos_inicial} para {a_pos_final}', (10, 300))
    display_text(screen, f'Total movidos: {move_index}', (10, 340))
    pygame.display.flip()

    return move_index

# Função para converter cromossomo para dataframe
def cromossomo_para_dataframe(melhor_cromossomo):
    data = {
        'Container': range(1, 31),
        'Posição Inicial': np.random.choice(range(30), 30, replace=False),
        'Posição Final': np.random.choice(range(30), 30, replace=False),
        'Ordem de Movimentação': np.random.choice(range(30), 30, replace=False)
    }
    return pd.DataFrame(data)

# Dados fictícios
melhor_cromossomo = None
df_cromossomo = cromossomo_para_dataframe(melhor_cromossomo)

# Inicializar o estado do porto e da balsa
porto = np.zeros((2, 5, 3), dtype=int)
balsa = np.zeros((3, 4, 3), dtype=int)

# Preencher o porto com os dados iniciais
ordem_inicial = df_cromossomo.sort_values(by='Posição Inicial')['Container'].tolist()
idx = 0
for andar in range(2):
    for x in range(5):
        for y in range(3):
            if idx < 30:
                porto[andar, x, y] = ordem_inicial[idx]
                idx += 1

# Loop principal do Pygame
running = True
move_next = False
move_index = 0
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:
            if BUTTON_POSITION[0] <= pygame.mouse.get_pos()[0] <= BUTTON_POSITION[0] + BUTTON_SIZE[0] and BUTTON_POSITION[1] <= pygame.mouse.get_pos()[1] <= BUTTON_POSITION[1] + BUTTON_SIZE[1]:
                move_next = True

    move_index = Simular_preencher_porto(df_cromossomo, porto, balsa, move_next, move_index)
    move_next = False

    # Desenhar botão
    hover = BUTTON_POSITION[0] <= pygame.mouse.get_pos()[0] <= BUTTON_POSITION[0] + BUTTON_SIZE[0] and BUTTON_POSITION[1] <= pygame.mouse.get_pos()[1] <= BUTTON_POSITION[1] + BUTTON_SIZE[1]
    draw_button(screen, BUTTON_POSITION, BUTTON_SIZE, 'Mover', hover)
    pygame.display.flip()

pygame.quit()


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import os



def Simular_preencher_porto(df_cromossomo):
    # Inicializar o porto e balsa
    porto = np.zeros((2, 5, 3), dtype=int)
    balsa = np.zeros((3, 4, 3), dtype=int)
    porto_preenchido = np.ones((2, 5, 3), dtype=int)
    
    # Preencher o porto com os dados do cromossomo
    ordem_inicial = []
    ordem_final = []

    df_cromossomo = df_cromossomo.sort_values(by='Posição Inicial')
    for _, linha in df_cromossomo.iterrows():
        ordem_inicial.append(linha['Container'])
    idx = 0
    for andar in range(2):
        for x in range(5):
            for y in range(3):
                if idx < 30:
                    porto[andar, x, y] = ordem_inicial[idx]
                    idx += 1

    df_cromossomo = df_cromossomo.sort_values(by='Ordem de Movimentação')

    # Diretório para salvar os frames
    save_dir = 'frames_porto_balsa'
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    frame_index = 0

    # Função para salvar o estado atual como imagem
    def save_frame(porto, balsa, frame_index):
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        axes[0].imshow(porto.reshape(-1, 3), cmap='gray', vmin=0, vmax=1)
        axes[0].set_title('Porto')
        axes[1].imshow(balsa.reshape(-1, 3), cmap='gray', vmin=0, vmax=1)
        axes[1].set_title('Balsa')
        plt.savefig(f'{save_dir}/frame_{frame_index}.png')
        plt.close()

    # Salvar estado inicial
    save_frame(porto_preenchido, balsa, frame_index)
    frame_index += 1

    # Realizar movimentações
    for _, linha in df_cromossomo.iterrows():
        container = linha['Container']
        a_pos_inicial = linha['Posição Inicial']
        a_pos_final = linha['Posição Final']
        pos_inicial = linha['Posição Inicial'] 
        pos_final = linha['Posição Final'] 

        andar_ini, x_ini, y_ini = pos_inicial // 15, (pos_inicial % 15) // 3, (pos_inicial % 15) % 3
        andar_fin, x_fin, y_fin = pos_final // 12, (pos_final % 12) // 3, (pos_final % 12) % 3

        # Verificar se podemos remover do porto preenchido
        pode_mover = True
        if andar_ini < 1:
            if porto_preenchido[andar_ini + 1, x_ini, y_ini] == 1:
                pode_mover = False

        # Verificar se podemos adicionar à balsa
        if andar_fin > 0:
            if balsa[andar_fin - 1, x_fin, y_fin] == 0:
                pode_mover = False

        if pode_mover:
            # Atualizar porto_preenchido e balsa
            porto_preenchido[andar_ini, x_ini, y_ini] = 0
            balsa[andar_fin, x_fin, y_fin] = 1
            
            print(f"Movendo container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")
            save_frame(porto_preenchido, balsa, frame_index)
            frame_index += 1
        else:
            print(f"não moveu container {container} de ({a_pos_inicial})({pos_inicial})({andar_ini}, {x_ini}, {y_ini}) para ({a_pos_final})({pos_final})({andar_fin}, {x_fin}, {y_fin})")

    # Criar o GIF
    frames = []
    for i in range(frame_index):
        frame = Image.open(f'{save_dir}/frame_{i}.png')
        frames.append(frame)

    frames[0].save('simulacao_porto_balsa.gif', format='GIF',
                   append_images=frames[1:], save_all=True, duration=500, loop=0)

Simular_preencher_porto(cromossomo_para_dataframe(melhor_cromossomo))


Movendo container 1 de (27)(27)(1, 4, 0) para (3)(3)(0, 1, 0)
Movendo container 19 de (15)(15)(1, 0, 0) para (0)(0)(0, 0, 0)
Movendo container 5 de (25)(25)(1, 3, 1) para (11)(11)(0, 3, 2)
Movendo container 10 de (23)(23)(1, 2, 2) para (8)(8)(0, 2, 2)
Movendo container 27 de (28)(28)(1, 4, 1) para (5)(5)(0, 1, 2)
Movendo container 24 de (13)(13)(0, 4, 1) para (10)(10)(0, 3, 1)
Movendo container 13 de (20)(20)(1, 1, 2) para (17)(17)(1, 1, 2)
Movendo container 29 de (19)(19)(1, 1, 1) para (4)(4)(0, 1, 1)
Movendo container 16 de (21)(21)(1, 2, 0) para (9)(9)(0, 3, 0)
Movendo container 23 de (16)(16)(1, 0, 1) para (1)(1)(0, 0, 1)
Movendo container 6 de (8)(8)(0, 2, 2) para (20)(20)(1, 2, 2)
Movendo container 25 de (12)(12)(0, 4, 0) para (6)(6)(0, 2, 0)
Movendo container 9 de (22)(22)(1, 2, 1) para (18)(18)(1, 2, 0)
Movendo container 18 de (1)(1)(0, 0, 1) para (13)(13)(1, 0, 1)
Movendo container 0 de (4)(4)(0, 1, 1) para (16)(16)(1, 1, 1)
Movendo container 14 de (10)(10)(0, 3, 1) para (21)(